# NOTICE
Caution: This notebook has not been tested in the artifact reproduction setting but will be useful as a reference point for future research -- be prepared to modify it. 

Use case: For researchers intending to run more experiments in a cluster setting.
Requirements: Brooce, redis.

You may contact the authors for further questions.

In [1]:
%run ../includes/common-20230414.ipynb

CPython 2024-01-16T11:09:31.175421


CPU times: user 884 µs, sys: 0 ns, total: 884 µs
Wall time: 1.12 ms


In [2]:
exp_common = dict(exp_date='20240101', group='fall24')
base_params = {
    **ef.csize_wrs_from_dwpds(csizes=[ef.DEFAULT_CSIZE], dwpds=[7.5, 15, 3.75, 1, 20]),
     **ef.traces(
         regions=[
             '201910/Region1',
             '201910/Region2',
             '201910/Region3',
             '202110/Region4',
             '20230325/Region5',
             '20230325/Region6',
             '20230325/Region7',
         ],
         # This requires more samples to be downloaded
         # ratios=[.1], max_samples=10, sample_start=0),
         ratios=[.1], max_samples=1, sample_start=0),
}

In [4]:
# Generates samples; requires full traces to be downloaded.
# local_cluster.ensure_samples(base_params, only_wait=False)

In [3]:
%%time
df_guess = pd.read_csv('../../data/results_release.csv.gz', low_memory=False)
all_exps = {}

CPU times: user 6.29 s, sys: 524 ms, total: 6.81 s
Wall time: 6.81 s


In [4]:
%cd ../..

/users/dlwong/projects/Baleen-FAST24


In [5]:
%%time
eff = ef.ExpFactory(name='alltraces', desc='All traces', **exp_common)
eff.add_params({**ef.EXP_OPT, **ef.prefetchs(ef.PF_BASIC), **base_params})
eff.add_params({**ef.EXP_STATIC, **ef.prefetchs(ef.PF_BASIC), **base_params})
eff.add_params({**ef.EXP_MLNEW, **ef.prefetchs(ef.PF_BASIC), **base_params})
eff.update(all_exps, df_guess=df_guess)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

CPU times: user 21.1 s, sys: 174 ms, total: 21.2 s
Wall time: 21.3 s


In [6]:
all_exps_ = list(flatten(all_exps.values()))
len(all_exps_), all_exps.keys()#, all_exps

(700, dict_keys(['20240101_alltraces']))

In [ ]:
# Useful if experiments failed
# monitor.relaunch(all_exps)

In [ ]:
monitor.save(all_exps)
monitor.run_exps(all_exps, relaunch_stale=True, queue='par4', top=False, displayer=None, limit_running_exps=400)